## Importing the CIFAR-100 dataset

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data(label_mode='fine')

print(x_train.shape, y_train.shape)

169001437/169001437 [==============================] - 6s 0us/step
(50000, 32, 32, 3) (50000, 1)


In [2]:
print(tf.config.list_physical_devices('GPU'))

[]


In [2]:
from keras.applications.inception_v3 import preprocess_input
from tensorflow.data import Dataset
from keras.layers import Resizing, Rescaling
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

print(x_train.shape)
#print(x_valid.shape)
print(x_test.shape)

x_train = np.moveaxis(x_train, -1, 1)
x_valid = np.moveaxis(x_valid, -1, 1)
x_test = np.moveaxis(x_test, -1, 1)

print(x_train.shape)
#print(x_train.shape)
print(x_test.shape)

enc = LabelBinarizer()

y_train = enc.fit_transform(y_train)
y_test = enc.fit_transform(y_test)

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2)

IMG_SIZE = 299

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
"""
train_ds = train_ds.map(
    lambda x,y: (preprocess_input(x), y)
)
valid_ds = valid_ds.map(
    lambda x,y: (preprocess_input(x), y)
)
test_ds = test_ds.map(
    lambda x,y: (preprocess_input(x), y)
)"""

resize_and_rescale = tf.keras.Sequential([
    Resizing(IMG_SIZE, IMG_SIZE),
    Rescaling(1./255, offset=-1),
])

aug_ds = train_ds.map(
  lambda x, y: (resize_and_rescale(x, training=True), y)
)


(50000, 32, 32, 3)
(10000, 32, 32, 3)


NameError: name 'x_valid' is not defined

In [2]:
from keras.applications.inception_v3 import preprocess_input
from tensorflow.data import Dataset
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.image import resize

enc = LabelBinarizer()

y_train = enc.fit_transform(y_train)
y_test = enc.fit_transform(y_test)

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2)

IMG_SIZE = 299

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_ds = train_ds.map(
    lambda x,y: (tf.image.resize(x, (299, 299)), y)
)
valid_ds = valid_ds.map(
    lambda x,y: (tf.image.resize(x, (299, 299)), y)
)
test_ds = test_ds.map(
    lambda x,y: (tf.image.resize(x, (299, 299)), y)
)

train_ds = train_ds.map(
    lambda x,y: (preprocess_input(x, data_format=), y)
)
valid_ds = valid_ds.map(
    lambda x,y: (preprocess_input(x), y)
)
test_ds = test_ds.map(
    lambda x,y: (preprocess_input(x), y)
)

In [9]:
print(train_ds)

<MapDataset element_spec=(TensorSpec(shape=(299, 299, 3), dtype=tf.float32, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>


## Inception v4 Implementation

In [10]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add own top to pre-trained model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

predictions = Dense(100, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# first train only the top layers
# freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy')




87910968/87910968 [==============================] - 3s 0us/step


In [12]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, TerminateOnNaN
import datetime

logdir = "./logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")



# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.
model.fit(train_ds,
          epochs=5,
          batch_size=1024,
          validation_data=valid_ds,
          validation_batch_size=1024,
          callbacks=[
              TensorBoard(
                  log_dir=logdir,
                  histogram_freq=1
              ),
              EarlyStopping(
                  monitor='val_top_1_accuracy',
                  patience=10,
                  restore_best_weights=True,
              ),
              ModelCheckpoint(
                  logdir+'/checkpoint',
                  monitor='val_top_1_accuracy',
                  save_best_only=True,
              ),
              TerminateOnNaN(),
          ],
)


Epoch 1/5


ValueError: in user code:

    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/opt/conda/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/input_spec.py", line 251, in assert_input_compatibility
        f'Input {input_index} of layer "{layer_name}" '

    ValueError: Ex

In [ ]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
model.compile(optimizer='adam', loss='categorical_crossentropy')
# run previous block to train model again, but with more epochs

In [ ]:
%load_external tensorboard

In [ ]:
%tensorboard --logdir ./logs/fit